In [6]:
from unstructured.partition import pdf, docx, pptx
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import unstructured
import os

In [2]:
with open('api_keys.txt') as f: api_keys = dict(key.split() for key in f.read().split('\n'))

In [3]:
api_keys['unstructured_api_key']

'RYrkDfIfLO1cb4Ltyaslqe10iJPQ01'

In [4]:
dir = 'docs/math138/'
elements = []

In [5]:
for filename in os.listdir(dir):
    filepath = f'{dir}{filename}'
    if os.path.isfile(filepath):
        match filename.split('.')[-1]:
            case 'pdf':
                elements += pdf.partition_pdf(
                    filename=filepath,
                    strategy='hi_res',
                    chunking_strategy='by_title',
                    extract_images_in_pdf=True,
                    infer_table_structure=True,
                    extract_image_block_types=['Image', 'Table'],
                    max_characters=1000,
                    new_after_n_chars=1500,
                    combine_text_under_n_chars=250
                    )
            case 'docx':
                elements += docx.partition_docx(
                    filename=filepath,
                    strategy='hi_res',
                    chunking_strategy='by_title',
                    extract_images_in_pdf=True,
                    infer_table_structure=True,
                    extract_image_block_types=['Image', 'Table'],
                    max_characters=1000,
                    new_after_n_chars=1500,
                    combine_text_under_n_chars=250
                    )
            case 'pptx':
                elements += pptx.partition_pptx(
                    filename=filepath,
                    strategy='hi_res',
                    chunking_strategy='by_title',
                    extract_images_in_pdf=True,
                    infer_table_structure=True,
                    extract_image_block_types=['Image', 'Table'],
                    max_characters=1000,
                    new_after_n_chars=1500,
                    combine_text_under_n_chars=250
                    )

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
elements[0]

In [31]:
documents = []
for element in elements:
    if isinstance(element, unstructured.documents.elements.CompositeElement): documents.append(Document(page_content=element.text, metadata=element.metadata.to_dict()))
    elif isinstance(element, unstructured.documents.elements.Table): pass # do something here

db = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

c:\Users\parth\anaconda3\envs\cs246helper\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.